<font size = 6><i>Classification de chiffres et comptage de mots</i></font>

Beaucoup de jeux de données sont de très grandes tailles, et faire des calculs dessus nécessite énormément de temps. Certains algorithmes sont décomposables en opérations élementaires, ce qui facilite la gestion des calculs assez lourds.

Le but du TP est de voir quelques méthodes qui simplifient les grands calculs sur des grosses bases de données.


# A-] Classification de chiffres
--

Pour montrer l'intérêt de la parallélisation, il faut d'abord télécharger un grand jeu de données.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
# tensorflow est une librairie qui est assez connue pour l'apprentissage statistique, notamment les réseaux de neurones
# on l'utilisera juste pour charger les données

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

/home/drone/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


11493376/11490434 [==============================] - 11s 1us/step


Ce jeu de données est une base de données d'images représentant des chiffres de 1 à 9 contenue dans x, accompagné d'une série de labels de "0" à "9" contenus dans y.

x_train[1] contient la première image sous forme de pixels (1 image = 28 pixels*28 pixels), y_train[1] le libellé du premier chiffre.

<b>1) Pour s'en convaincre, affichez les premières images à l'aide de la fonction imshow() de matplotlib</b>

<b>2) Par exemple, cherchez la probabilité de tomber sur le chiffre 1 dans y_train, et calculez le temps pris par votre calcul à  l'aide de la fonction clock() de la librairie time.</b>

<b>3) Cette instruction tourne plutôt vite. Comment se comporte le temps quand on fait une grosse opération? Faites une analyse en composantes principales avec le package PCA, et affichez les points à l'aide de matplotlib.</b>
    
Il peut être utile de formater les données dans un premier temps, de sorte qu'une image soit écrite dans une seule ligne dans votre base de données, et pas dans un tableau de 28*28. Pour cela, pensez à utiliser la fonction reshape de numpy.

Si vous vous sentez à l'aise: tracer des ellipses autour des groupes de points pour "marquer les limites" des groupes.

<b> 4) Maintenant, on va comparer le temps d'éxécution d'une opération lancée sur un coeur , au temps d'exécution de la même opération répartie entre plusieurs coeurs. Pour cela , on va utiliser la fonction K-means, vu dans le tp précédent, qui fait de la classification des individus (ici de la classification des chiffres).</b>

Dans la logique, il faudrait penser à faire 10 groupes de points, pour que chaque groupe isolé par les kmeans correspondent à un chiffre. Prenez le réflexe d'aller fouiller dans la doc, en l'occurence dans ce cas-là: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


 

Cette fonction dipose d'une option n_jobs qui permet de partager les calculs entre plusieurs coeurs. A priori, le nombre maximal pour n_jobs est le nombre de coeurs que possède votre ordinateur. Pour déterminer le nombre de coeurs maximal:
- https://www.it-connect.fr/afficher-le-nombre-de-coeurs-sous-linux/ pour linux 
- https://support.microsoft.com/fr-fr/help/4026757/windows-10-find-out-how-many-cores-your-processor-has pour windows

Testez un algorithme de KMeans lancé avec l'option n_jobs = 1, puis le même algo avec n_jobs = votre nombre de coeurs, et comparez les temps d'exécution.


Je vous conseille de lancer un htop depuis linux pour voir comment les coeurs sont utilisés par l'algorithme suivant les différentes configurations (ou le gestionnaire des tâches sous windows).

Ne prenez que quelques milliers de ligne pour démarrer, sinon l'algorithme prendra trop de temps (10 000 max).

Pour un seul job

La version à quatre jobs

<b> 5) Si vous avez le temps, essayez de faire varier n_jobs et et le nombre de lignes. Puis représentez graphiquement tous les temps d'exécution sur un même graphe, en fonction de n_jobs.
    
Sinon, passez directement au B-].</b>

Cet algorithme est sympathique, au sens où il gère pour vous toute la partie décomposition des opérations. Si vous voulez répartir les opérations sur plusieurs coeurs, il faut dire manuellement à chaque coeur de quoi il doit s'occuper à l'aide de threads. 

Voir https://openclassrooms.com/fr/courses/235344-apprenez-a-programmer-en-python/2235545-la-programmation-parallele-avec-threading si vous souhaitez aller plus loin sur le sujet. 

(Bonus)
<b> 6) Introduction au machine learning : Utilisez une regression logistique pour prédire les chiffres représentés sur lessur les 10000 images de test à partir de 5000 images issues de l'échantillon d'apprentissage. Combien avez-vous d'erreurs? </b>

On pourra utiliser LogisticRegression, de sklearn.linear_model.

# B-] Comptage de mots avec Spark


C'est quoi spark? C'est un environnement de calculs distribués. Il permet, à l'image des questions précédentes, de décomposer les gros calculs en une multitude de petits calculs qui s'éxécutent en parallèle.

Pour l'installation, qui n'est pas évidente à suivre:
http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx2/td_3a_spark.html

In [2]:
Ce code ne doit être exécuté qu'une fois. La variable sc est alors notre interface spark, qui tourne en local.import findspark
findspark.init("C:\\Users\\llesoil\\Downloads\\spark") # à remplacer par votre SPARK_HOME, l'emplacement de spark
import pyspark
sc = pyspark.SparkContext(appName="TP3")

Ce code ne doit être exécuté qu'une fois. La variable sc est alors notre interface spark, qui tourne en local.

In [3]:
s = 'ils ont des chapeaux ronds '+ 'vive la Bretagne '+'ils ont des chapeaux ronds '+ 'vive les bretons' 
sc.parallelize(s.split()).map(lambda word: (word, 1)).reduceByKey(lambda a,b:a+b).collect()

[('la', 1),
 ('ronds', 2),
 ('Bretagne', 1),
 ('les', 1),
 ('chapeaux', 2),
 ('vive', 2),
 ('ils', 2),
 ('ont', 2),
 ('bretons', 1),
 ('des', 2)]

Le [schéma](https://github.com/llesoil/modelisation_des_problemes_scientifiques-/blob/master/Aides/mapreduce.png), tiré de ce [tuto](https://nyu-cds.github.io/python-bigdata/02-mapreduce/) explique bien le principe du mapreduce, un algorithme utilisé par spark qui permet de distribuer des calculs.:

- On répartit les données entre les différentes machines (s'il y a quatre machines, on peut supposer que chacune s'occupe d'une phrase dans notre cas), c'est le mapping
- Chacun récupère les mots de sa phrase à partir de la chaine de caractère (le .split() découpe la chaine suivant les espaces)
- Chacun compte ses mots individuellement, c'est le map
- Puis on passe à la phase reduce, on additionne tous les comptes des mots associés aux quatre phrases (reducebKey)
- Enfin, on peut afficher les résultats du dessous

L'intérêt, c'est que <B>toute la partie mapping peut se faire en parallèle</B>. 

Si vous avez un roman à analyser, ça sera plus rapide si chaque ordinateur s'occupe d'un chapitre, comparé à une analyse séquentielle.

In [4]:
import os

path = os.getcwd() + "//"+"loutre_des_mers.txt" 
# os.getcwd() donne l'adresse du répertoire courant
# Vous pouvez télécharger le fichier loutre_des_mers.txt et mettre l'adresse locale dans la variable path

file = sc.textFile(path)
# Crée le fichier à partir du chemin

output = file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b).collect()

# on affiche les dix premiers mots
for (word, count) in output[0:10]:
        print("%s: %i" % (word, count))

loutre: 6
mer: 6
lutris): 1
est: 2
des: 6
must�lid�s): 1
vivant: 1
dans: 3
le: 3
du: 5


Le code fonctionne également avec un fichier texte.

<b>1) Appliquez le même code au fichier twitter_trump.txt, qui contient des tweets concernant/de Donald Trump pris sur http://www.trumptwitterarchive.com/archive. Quels sont les mots qui reviennent le plus fréquemement? Est-ce Barack Obama ou Hillary Clinton qui est le.a plus cité.e dans ce fichier?</b>

Indications:
- On pourra utiliser des dictionnaires, la fonction sorted() pourra sans doute vous aider. 
- Si vous voulez de meilleurs résultats, vous pouvez utiliser la fonction lower() sur chacun des mots pour enlever les majuscules, et le module stop_words pour ne pas tenir compte des mots les plus couramment utilisés en anglais.

<b> 2) Estimation de la constante pi à l'aide de Spark<b>